# Pipeline: Silver -> Gold

## Instruções e informações

### Objetivo:  

Este notebook realiza a extração, transformação e carga (ETL) dos dados da camada **Silver**  para a camada **Gold** (modelo dimensional Star Schema no schema `dw`).

### Configuração Inicial

Esta seção realiza a configuração inicial de todo o ambiente que será utilizado nas análises subsequentes, incluindo:  

- Importação das bibliotecas necessárias (ex.: `pandas`, `numpy`, `matplotlib`, `seaborn`), explicando a função de cada uma na sequência de transformações.  
- Definição dos caminhos para os arquivos de dados brutos (CSV da camada Bronze) que serão processados.  
- Identificação de metadados relevantes que podem ser úteis para a limpeza e análise, como tipos de colunas, valores nulos, formatos de data, etc.  


In [17]:
import os
import re
from sqlalchemy import create_engine
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lower, trim, regexp_replace, coalesce, lit, floor, to_date, year, month, dayofmonth, concat_ws, date_format
from pyspark.sql.types import IntegerType, DoubleType, StringType, DateType
import pandas as pd
import psycopg2
from psycopg2 import sql
import psycopg2.extras
from datetime import datetime

spark = (
    SparkSession.builder
    .appName("Formula1Analysis")
    .master("local[*]") 
    .config("spark.driver.memory", "4g") 
    .config("spark.executor.memory", "4g") 
    .getOrCreate()
)

if spark.sparkContext.appName == "Formula1Analysis":
    print("✅ Configuração do PySpark concluída com sucesso.")
    print(f"Versão do Spark: {spark.version}")
else:
    print("❌ Erro na configuração do PySpark.")


# Configurações de conexão com o banco de dados
DB_CONFIG = {
    'host': 'localhost',      # Ou '127.0.0.1' - Deve ser 'localhost' se o Docker for exposto localmente.
    'port': 5432,             # Porta padrão do PostgreSQL, assumindo que você a expôs assim.
    'database': 'f1_dw',      # <-- Ajustado para POSTGRES_DB
    'user': 'user_f1',        # <-- Ajustado para POSTGRES_USER
    'password': 'password_f1' # <-- Ajustado para POSTGRES_PASSWORD
}

print("✓ Bibliotecas importadas com sucesso!")
print(f"✓ Configuração: {DB_CONFIG['database']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}")

# Estabelecer conexão única que será reutilizada
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("✓ Conexão estabelecida com o banco de dados")
    print(f"✓ Database: {DB_CONFIG['database']}")
    print(f"✓ Host: {DB_CONFIG['host']}:{DB_CONFIG['port']}")
except psycopg2.OperationalError as e:
    print(f"✗ ERRO DE CONEXÃO: Não foi possível conectar ao banco de dados.")
    print(f"   Verifique se o container PostgreSQL está rodando na porta 5432 e se as credenciais estão corretas.")
    print(f"   Detalhes do erro: {e}")

✅ Configuração do PySpark concluída com sucesso.
Versão do Spark: 4.0.1
✓ Bibliotecas importadas com sucesso!
✓ Configuração: f1_dw@localhost:5432
✓ Conexão estabelecida com o banco de dados
✓ Database: f1_dw
✓ Host: localhost:5432


___

### Criação do Schema dw e Todas as Tabelas (usando ddl.sql)

In [19]:
ddl_file_path = '../data_layer/gold/ddl.sql' 

print("\nIniciando a criação do Schema DW e Tabelas:")
print("=" * 60)

try:
    with open(ddl_file_path, 'r', encoding='utf-8') as file:
        ddl_script = file.read()
    
    print(f"✓ Arquivo DDL carregado: {ddl_file_path}")
    print(f"✓ Tamanho do script: {len(ddl_script)} caracteres\n")
    
    cursor.execute(ddl_script)
    conn.commit()
    
    print("=" * 60)
    print("✓ Schema 'dw' criado e recriado com sucesso (DROP/CREATE)!")
    print("✓ Tabelas dimensionais e fato criadas:")
    print("  - dw.DIM_PILOTO, dw.DIM_EQUIPE, dw.DIM_CIRCUITO, dw.DIM_STATUS, dw.DIM_CORRIDA")
    print("  - dw.FAT_RESULTADOS")
    print("✓ Chaves estrangeiras configuradas (ON DELETE RESTRICT)")
    print("=" * 60)
    
except FileNotFoundError:
    print(f"✗ Erro: Arquivo DDL não encontrado em '{ddl_file_path}'")
    print("  Certifique-se de que o caminho está correto.")

except psycopg2.Error as e:
    print(f"✗ Erro ao executar DDL (PostgreSQL Error): {e}")
    conn.rollback() # Desfaz quaisquer alterações se houver erro
    
except Exception as e:
    print(f"✗ Erro inesperado ao executar DDL: {e}")
    conn.rollback()


Iniciando a criação do Schema DW e Tabelas:
✓ Arquivo DDL carregado: ../data_layer/gold/ddl.sql
✓ Tamanho do script: 1816 caracteres

✓ Schema 'dw' criado e recriado com sucesso (DROP/CREATE)!
✓ Tabelas dimensionais e fato criadas:
  - dw.DIM_PILOTO, dw.DIM_EQUIPE, dw.DIM_CIRCUITO, dw.DIM_STATUS, dw.DIM_CORRIDA
  - dw.FAT_RESULTADOS
✓ Chaves estrangeiras configuradas (ON DELETE RESTRICT)


___
### Extração de Dados da Camada Silver 

Inicialmente vamos puxar as informações necessárias para montar a tabela fato, a partir das tabelas presentes no schema "silver"


In [25]:
query_fato = """
SELECT
    -- PK do FATO
    rs.result_id AS resultado_pk,
    
    -- FKs para as Dimensões (Chaves Substitutas)
    rs.race_id AS srk_corrida_fk,
    rs.driver_id AS srk_piloto_fk,
    rs.constructor_id AS srk_equipe_fk,
    rs.status_id AS srk_status_fk,
    
    -- MÉTRICAS (Fatos a serem agregados)
    rs.grid,
    rs.position_order,
    rs.points,
    rs.laps,
    rs.milliseconds,
    rs.rank
    
FROM 
    silver.results_silver rs
WHERE
    rs.race_id IS NOT NULL 
"""

# Armazena o resultado no DataFrame solicitado
df_silver_fato_resultados = pd.read_sql_query(query_fato, conn)

print("✓ Carregamento para o Fato isolado concluído.")
print(f"✓ DataFrame 'df_silver_fato_resultados' criado com {len(df_silver_fato_resultados)} registros.")
print(f"✓ Colunas: {list(df_silver_fato_resultados.columns)}")

print("\n Preview dos dados do Fato:")
display(df_silver_fato_resultados.head())

✓ Carregamento para o Fato isolado concluído.
✓ DataFrame 'df_silver_fato_resultados' criado com 26759 registros.
✓ Colunas: ['resultado_pk', 'srk_corrida_fk', 'srk_piloto_fk', 'srk_equipe_fk', 'srk_status_fk', 'grid', 'position_order', 'points', 'laps', 'milliseconds', 'rank']

 Preview dos dados do Fato:


/tmp/ipykernel_178440/2236170282.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_silver_fato_resultados = pd.read_sql_query(query_fato, conn)


,resultado_pk,srk_corrida_fk,srk_piloto_fk,srk_equipe_fk,srk_status_fk,grid,position_order,points,laps,milliseconds,rank
0,1,18,1,1,1,1,1,10.0,58,5690616.0,2.0
1,3,18,3,3,1,7,3,6.0,58,5698779.0,5.0
2,5,18,5,1,1,3,5,4.0,58,5708630.0,1.0
3,6,18,6,3,11,13,6,3.0,57,NaN,14.0
4,12,18,12,4,8,20,12,0.0,30,NaN,16.0


___
### Extração dos dados necessários - Tabela Dimensão - DIM_CORRIDA

In [24]:
query_dim_corrida = """
SELECT
    -- Chave Primária Substituta (SRK)
    r.race_id AS corrida_pk, 
    
    -- Atributos Contextuais
    r.year,
    r.round,
    r.circuit_id, 
    r.name,
    r.date,
    r.time
    
FROM 
    silver.races_silver r
WHERE
    r.race_id IS NOT NULL -- Garantia de integridade da chave
"""

df_dim_corrida = pd.read_sql_query(query_dim_corrida, conn)


df_dim_corrida = df_dim_corrida.drop_duplicates(subset=['corrida_pk'])

print("✓ Extração para DIM_CORRIDA concluída.")
print(f"✓ DataFrame 'df_dim_corrida' criado com {len(df_dim_corrida)} registros.")
print(f"✓ Colunas: {list(df_dim_corrida.columns)}")
print("\n Preview da Dimensão Corrida:")
display(df_dim_corrida.head())

✓ Extração para DIM_CORRIDA concluída.
✓ DataFrame 'df_dim_corrida' criado com 1125 registros.
✓ Colunas: ['corrida_pk', 'year', 'round', 'circuit_id', 'name', 'date', 'time']

 Preview da Dimensão Corrida:


/tmp/ipykernel_178440/4229273481.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dim_corrida = pd.read_sql_query(query_dim_corrida, conn)


,corrida_pk,year,round,circuit_id,name,date,time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,2025-11-25 06:00:00
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,2025-11-25 09:00:00
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,2025-11-25 07:00:00
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,2025-11-25 12:00:00
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,2025-11-25 12:00:00


### Populando no banco - Tabela Dimensão - DIM_CORRIDA

In [ ]:
TARGET_TABLE = "dw.DIM_CORRIDA"

COLUMNS = [
    'corrida_pk', 'year', 'round', 'circuit_id', 'name', 'date', 'time'
]

print(f"\n--- INICIANDO CARGA DA DIMENSÃO: {TARGET_TABLE} ---")

try:
    cursor.execute(f"TRUNCATE TABLE {TARGET_TABLE} RESTART IDENTITY CASCADE;")
    print(f"✓ Tabela {TARGET_TABLE} truncada para recarga.")

    data_to_insert = df_dim_corrida[COLUMNS].to_records(index=False).tolist()
    insert_query = f"""
        INSERT INTO {TARGET_TABLE} ({', '.join(COLUMNS)}) 
        VALUES %s
        ON CONFLICT (corrida_pk) DO NOTHING; -- Evita erro se houver duplicatas
    """
    
    psycopg2.extras.execute_values(
        cursor,
        insert_query,
        data_to_insert,
        page_size=10000 
    )
    
    conn.commit()
    print(f"✓ {len(data_to_insert)} registros inseridos com sucesso em {TARGET_TABLE}.")
    
except psycopg2.Error as e:
    print(f"✗ ERRO CATASTRÓFICO NO POSTGRES DURANTE A CARGA DA DIMENSÃO: {e}")
    conn.rollback() 

print("--- ✅ CARGA DA DIMENSÃO CORRIDA CONCLUÍDA ---")


--- ⏳ INICIANDO CARGA DA DIMENSÃO: dw.DIM_CORRIDA ---
✓ Tabela dw.DIM_CORRIDA truncada para recarga.
✓ 1125 registros inseridos com sucesso em dw.DIM_CORRIDA.
--- ✅ CARGA DA DIMENSÃO CORRIDA CONCLUÍDA ---


___
### Extração dos dados necessários - Tabela Dimensão - DIM_CIRCUITO


In [ ]:
query_dim_circuito = """
SELECT
    -- Chave Primária Substituta (PK)
    c.circuit_id AS circuito_pk, 
    
    -- Atributos Contextuais
    c.circuit_ref,
    c.name,
    c.location,
    c.country,
    c.lat,
    c.lng,
    c.alt
    
FROM 
    silver.circuits_silver c
WHERE
    c.circuit_id IS NOT NULL 
"""

df_dim_circuito = pd.read_sql_query(query_dim_circuito, conn)
df_dim_circuito = df_dim_circuito.drop_duplicates(subset=['circuito_pk'])

print("✓ Extração para DIM_CIRCUITO concluída.")
print(f"✓ DataFrame 'df_dim_circuito' criado com {len(df_dim_circuito)} registros.")
print(f"✓ Colunas: {list(df_dim_circuito.columns)}")
print("\n Preview da Dimensão Circuito:")
display(df_dim_circuito.head())

✓ Extração para DIM_CIRCUITO concluída.
✓ DataFrame 'df_dim_circuito' criado com 77 registros.
✓ Colunas: ['circuito_pk', 'circuit_ref', 'name', 'location', 'country', 'lat', 'lng', 'alt']

 Preview da Dimensão Circuito:


/tmp/ipykernel_178440/196820678.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dim_circuito = pd.read_sql_query(query_dim_circuito, conn)


,circuito_pk,circuit_ref,name,location,country,lat,lng,alt
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130


### Populando no banco - Tabela Dimensão - DIM_CORRIDA

In [29]:
TARGET_TABLE = "dw.DIM_CIRCUITO"

COLUMNS = [
    'circuito_pk', 'circuit_ref', 'name', 'location', 'country', 'lat', 'lng', 'alt'
]

print(f"\n--- INICIANDO CARGA DA DIMENSÃO: {TARGET_TABLE} ---")

try:
    cursor.execute(f"TRUNCATE TABLE {TARGET_TABLE} RESTART IDENTITY CASCADE;")
    print(f"✓ Tabela {TARGET_TABLE} truncada para recarga.")

    data_to_insert = df_dim_circuito[COLUMNS].to_records(index=False).tolist()

    insert_query = f"""
        INSERT INTO {TARGET_TABLE} ({', '.join(COLUMNS)}) 
        VALUES %s
        ON CONFLICT (circuito_pk) DO NOTHING;
    """
    
    psycopg2.extras.execute_values(
        cursor,
        insert_query,
        data_to_insert,
        page_size=10000 
    )
    
    conn.commit()
    print(f"✓ {len(data_to_insert)} registros inseridos com sucesso em {TARGET_TABLE}.")
    
except psycopg2.Error as e:
    print(f" ERRO NO POSTGRES DURANTE A CARGA DA DIMENSÃO: {e}")
    conn.rollback() 

print("--- ✅ CARGA DA DIMENSÃO CIRCUITO CONCLUÍDA ---")


--- INICIANDO CARGA DA DIMENSÃO: dw.DIM_CIRCUITO ---
✓ Tabela dw.DIM_CIRCUITO truncada para recarga.
✓ 77 registros inseridos com sucesso em dw.DIM_CIRCUITO.
--- ✅ CARGA DA DIMENSÃO CIRCUITO CONCLUÍDA ---


___
### Extração dos dados necessários - Tabela Dimensão - DIM_EQUIPE


In [30]:
query_dim_equipe = """
SELECT
    -- Chave Primária Substituta (PK)
    c.constructor_id AS equipe_pk, 
    
    -- Atributos Contextuais
    c.constructor_ref,
    c.name,
    c.nationality
    
FROM 
    silver.constructors_silver c
WHERE
    c.constructor_id IS NOT NULL 
"""

df_dim_equipe = pd.read_sql_query(query_dim_equipe, conn)
df_dim_equipe = df_dim_equipe.drop_duplicates(subset=['equipe_pk'])

print("✓ Extração para DIM_EQUIPE concluída.")
print(f"✓ DataFrame 'df_dim_equipe' criado com {len(df_dim_equipe)} registros.")
print(f"✓ Colunas: {list(df_dim_equipe.columns)}")
print("\n Preview da Dimensão Equipe:")
display(df_dim_equipe.head())

✓ Extração para DIM_EQUIPE concluída.
✓ DataFrame 'df_dim_equipe' criado com 212 registros.
✓ Colunas: ['equipe_pk', 'constructor_ref', 'name', 'nationality']

 Preview da Dimensão Equipe:


/tmp/ipykernel_178440/1897385856.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dim_equipe = pd.read_sql_query(query_dim_equipe, conn)


,equipe_pk,constructor_ref,name,nationality
0,1,mclaren,McLaren,British
1,2,bmw_sauber,BMW Sauber,German
2,3,williams,Williams,British
3,4,renault,Renault,French
4,5,toro_rosso,Toro Rosso,Italian


### Populando no banco - Tabela Dimensão - DIM_EQUIPE

In [31]:
TARGET_TABLE = "dw.DIM_EQUIPE"

COLUMNS = [
    'equipe_pk', 'constructor_ref', 'name', 'nationality'
]

print(f"\n--- INICIANDO CARGA DA DIMENSÃO: {TARGET_TABLE} ---")

try:
    cursor.execute(f"TRUNCATE TABLE {TARGET_TABLE} RESTART IDENTITY CASCADE;")
    print(f"✓ Tabela {TARGET_TABLE} truncada para recarga.")

    data_to_insert = df_dim_equipe[COLUMNS].to_records(index=False).tolist()

    insert_query = f"""
        INSERT INTO {TARGET_TABLE} ({', '.join(COLUMNS)}) 
        VALUES %s
        ON CONFLICT (equipe_pk) DO NOTHING;
    """
    
    psycopg2.extras.execute_values(
        cursor,
        insert_query,
        data_to_insert,
        page_size=10000 
    )
    
    conn.commit()
    print(f"✓ {len(data_to_insert)} registros inseridos com sucesso em {TARGET_TABLE}.")
    
except psycopg2.Error as e:
    print(f"✗ ERRO NO POSTGRES DURANTE A CARGA DA DIMENSÃO: {e}")
    conn.rollback() 

print("--- ✅ CARGA DA DIMENSÃO EQUIPE CONCLUÍDA ---")


--- INICIANDO CARGA DA DIMENSÃO: dw.DIM_EQUIPE ---
✓ Tabela dw.DIM_EQUIPE truncada para recarga.
✓ 212 registros inseridos com sucesso em dw.DIM_EQUIPE.
--- ✅ CARGA DA DIMENSÃO EQUIPE CONCLUÍDA ---


___
### Extração dos dados necessários - Tabela Dimensão - DIM_PILOTO


In [32]:
query_dim_piloto = """
SELECT
    -- Chave Primária Substituta (PK)
    dr.driver_id AS piloto_pk, 
    
    -- Atributos Contextuais
    dr.driver_ref,
    dr.code,
    dr.forename,
    dr.surname,
    dr.dob,
    dr.nationality
    
FROM 
    silver.drivers_silver dr
WHERE
    dr.driver_id IS NOT NULL 
"""

df_dim_piloto = pd.read_sql_query(query_dim_piloto, conn)
df_dim_piloto = df_dim_piloto.drop_duplicates(subset=['piloto_pk'])

print("✓ Extração para DIM_PILOTO concluída.")
print(f"✓ DataFrame 'df_dim_piloto' criado com {len(df_dim_piloto)} registros.")
print(f"✓ Colunas: {list(df_dim_piloto.columns)}")
print("\n Preview da Dimensão Piloto:")
display(df_dim_piloto.head())

✓ Extração para DIM_PILOTO concluída.
✓ DataFrame 'df_dim_piloto' criado com 861 registros.
✓ Colunas: ['piloto_pk', 'driver_ref', 'code', 'forename', 'surname', 'dob', 'nationality']

 Preview da Dimensão Piloto:


/tmp/ipykernel_178440/895900907.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dim_piloto = pd.read_sql_query(query_dim_piloto, conn)


,piloto_pk,driver_ref,code,forename,surname,dob,nationality
0,1,hamilton,HAM,Lewis,Hamilton,1985-01-07,British
1,2,heidfeld,HEI,Nick,Heidfeld,1977-05-10,German
2,3,rosberg,ROS,Nico,Rosberg,1985-06-27,German
3,4,alonso,ALO,Fernando,Alonso,1981-07-29,Spanish
4,5,kovalainen,KOV,Heikki,Kovalainen,1981-10-19,Finnish


### Populando no banco - Tabela Dimensão - DIM_PILOTO

In [33]:
TARGET_TABLE = "dw.DIM_PILOTO"

COLUMNS = [
    'piloto_pk', 'driver_ref', 'code', 'forename', 'surname', 'dob', 'nationality'
]

print(f"\n--- INICIANDO CARGA DA DIMENSÃO: {TARGET_TABLE} ---")

try:
    cursor.execute(f"TRUNCATE TABLE {TARGET_TABLE} RESTART IDENTITY CASCADE;")
    print(f"✓ Tabela {TARGET_TABLE} truncada para recarga.")

    data_to_insert = df_dim_piloto[COLUMNS].to_records(index=False).tolist()

    insert_query = f"""
        INSERT INTO {TARGET_TABLE} ({', '.join(COLUMNS)}) 
        VALUES %s
        ON CONFLICT (piloto_pk) DO NOTHING;
    """
    
    psycopg2.extras.execute_values(
        cursor,
        insert_query,
        data_to_insert,
        page_size=10000 
    )
    
    conn.commit()
    print(f"✓ {len(data_to_insert)} registros inseridos com sucesso em {TARGET_TABLE}.")
    
except psycopg2.Error as e:
    print(f"✗ ERRO NO POSTGRES DURANTE A CARGA DA DIMENSÃO: {e}")
    conn.rollback() 

print("--- ✅ CARGA DA DIMENSÃO PILOTO CONCLUÍDA ---")


--- INICIANDO CARGA DA DIMENSÃO: dw.DIM_PILOTO ---
✓ Tabela dw.DIM_PILOTO truncada para recarga.
✓ 861 registros inseridos com sucesso em dw.DIM_PILOTO.
--- ✅ CARGA DA DIMENSÃO PILOTO CONCLUÍDA ---


### Extração dos dados necessários - Tabela Dimensão - DIM_STATUS


In [34]:
query_dim_status = """
SELECT
    -- Chave Primária Substituta (PK)
    s.status_id AS status_pk, 
    
    -- Atributos Contextuais
    s.status
    
FROM 
    silver.status_silver s
WHERE
    s.status_id IS NOT NULL 
"""

df_dim_status = pd.read_sql_query(query_dim_status, conn)
df_dim_status = df_dim_status.drop_duplicates(subset=['status_pk'])

print("✓ Extração para DIM_STATUS concluída.")
print(f"✓ DataFrame 'df_dim_status' criado com {len(df_dim_status)} registros.")
print(f"✓ Colunas: {list(df_dim_status.columns)}")
print("\n Preview da Dimensão Status:")
display(df_dim_status.head())

✓ Extração para DIM_STATUS concluída.
✓ DataFrame 'df_dim_status' criado com 139 registros.
✓ Colunas: ['status_pk', 'status']

 Preview da Dimensão Status:


/tmp/ipykernel_178440/3175447733.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dim_status = pd.read_sql_query(query_dim_status, conn)


,status_pk,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


### Populando no banco - Tabela Dimensão - DIM_STATUS

In [ ]:
TARGET_TABLE = "dw.DIM_STATUS"

COLUMNS = [
    'status_pk', 'status'
]

print(f"\n--- INICIANDO CARGA DA DIMENSÃO: {TARGET_TABLE} ---")

try:
    cursor.execute(f"TRUNCATE TABLE {TARGET_TABLE} RESTART IDENTITY CASCADE;")
    print(f"✓ Tabela {TARGET_TABLE} truncada para recarga.")

    data_to_insert = df_dim_status[COLUMNS].to_records(index=False).tolist()

    insert_query = f"""
        INSERT INTO {TARGET_TABLE} ({', '.join(COLUMNS)}) 
        VALUES %s
        ON CONFLICT (status_pk) DO NOTHING;
    """
    
    psycopg2.extras.execute_values(
        cursor,
        insert_query,
        data_to_insert,
        page_size=10000 
    )
    
    conn.commit()
    print(f"✓ {len(data_to_insert)} registros inseridos com sucesso em {TARGET_TABLE}.")
    
except psycopg2.Error as e:
    print(f"✗ ERRO NO POSTGRES DURANTE A CARGA DA DIMENSÃO: {e}")
    conn.rollback() 

print("--- ✅ CARGA DA DIMENSÃO STATUS CONCLUÍDA ---")


--- ⏳ INICIANDO CARGA DA DIMENSÃO: dw.DIM_STATUS ---
✓ Tabela dw.DIM_STATUS truncada para recarga.
✓ 139 registros inseridos com sucesso em dw.DIM_STATUS.
--- ✅ CARGA DA DIMENSÃO STATUS CONCLUÍDA ---


___
### Extração dos dados necessários - Tabela Fato FAT_RESULTADOS

In [ ]:
query_fato = """
SELECT
    -- PK do FATO
    rs.result_id AS resultado_pk,
    
    -- FKs para as Dimensões (Chaves Substitutas)
    rs.race_id AS srk_corrida_fk,
    rs.driver_id AS srk_piloto_fk,
    rs.constructor_id AS srk_equipe_fk,
    rs.status_id AS srk_status_fk,
    
    -- MÉTRICAS (Fatos a serem agregados)
    rs.grid,
    rs.position_order,
    rs.points,
    rs.laps,
    rs.milliseconds,
    rs.rank
    
FROM 
    silver.results_silver rs
WHERE
    rs.race_id IS NOT NULL 
"""

df_silver_fato_resultados = pd.read_sql_query(query_fato, conn)

print("✓ Extração dos dados brutos do Fato concluída.")
print(f"✓ DataFrame 'df_silver_fato_resultados' criado com {len(df_silver_fato_resultados)} registros.")
print(f"✓ Colunas: {list(df_silver_fato_resultados)}")
print("\n Preview da Dimensão Status:")
display(df_silver_fato_resultados.head())

✓ Extração dos dados brutos do Fato concluída.
✓ DataFrame 'df_silver_fato_resultados' criado com 26759 registros.
✓ Colunas: ['resultado_pk', 'srk_corrida_fk', 'srk_piloto_fk', 'srk_equipe_fk', 'srk_status_fk', 'grid', 'position_order', 'points', 'laps', 'milliseconds', 'rank']

 Preview da Dimensão Status:


/tmp/ipykernel_178440/2271334141.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_silver_fato_resultados = pd.read_sql_query(query_fato, conn)


,resultado_pk,srk_corrida_fk,srk_piloto_fk,srk_equipe_fk,srk_status_fk,grid,position_order,points,laps,milliseconds,rank
0,1,18,1,1,1,1,1,10.0,58,5690616.0,2.0
1,3,18,3,3,1,7,3,6.0,58,5698779.0,5.0
2,5,18,5,1,1,3,5,4.0,58,5708630.0,1.0
3,6,18,6,3,11,13,6,3.0,57,NaN,14.0
4,12,18,12,4,8,20,12,0.0,30,NaN,16.0


### Populando no banco - Tabela Fato - FAT_RESULTADOS

In [39]:
TARGET_TABLE = "dw.FAT_RESULTADOS"
COLUMNS = [
    'resultado_pk', 'srk_corrida_fk', 'srk_piloto_fk', 'srk_equipe_fk', 
    'srk_status_fk', 'grid', 'position_order', 'points', 
    'laps', 'milliseconds', 'rank'
]

BIGINT_COLS_TO_CLEAN = [
    'resultado_pk', 'srk_corrida_fk', 'srk_piloto_fk', 
    'srk_equipe_fk', 'srk_status_fk', 'milliseconds'
]

print(f"\n--- INICIANDO CARGA FINAL DA TABELA FATO: {TARGET_TABLE} ---")

try:
    for col_name in BIGINT_COLS_TO_CLEAN:
        df_silver_fato_resultados[col_name] = df_silver_fato_resultados[col_name].fillna(0).round(0).astype('int64')

    cursor.execute(f"TRUNCATE TABLE {TARGET_TABLE} RESTART IDENTITY CASCADE;")
    print(f"✓ Tabela {TARGET_TABLE} truncada para recarga.")

    data_to_insert = df_silver_fato_resultados[COLUMNS].to_records(index=False).tolist()

    insert_query = f"""
        INSERT INTO {TARGET_TABLE} ({', '.join(COLUMNS)}) 
        VALUES %s
    """
    
    psycopg2.extras.execute_values(
        cursor,
        insert_query,
        data_to_insert,
        page_size=10000 
    )
    
    # 5. Commit e Relatório
    conn.commit()
    print(f"✓ {len(data_to_insert)} registros inseridos com sucesso em {TARGET_TABLE}.")
    
except psycopg2.IntegrityError as e:
    print(f"✗ ERRO DE INTEGRIDADE (FK VIOLATION): A carga falhou. DETALHE: {e.diag.message_detail}")
    print("  Verifique se TODAS as tabelas Dimensão foram carregadas antes do Fato.")
    conn.rollback() 
except Exception as e:
    print(f"✗ ERRO INESPERADO NA EXECUÇÃO: {e}")
    conn.rollback()

print("--- ✅ CARGA DO STAR SCHEMA FINALIZADA ---")


--- INICIANDO CARGA FINAL DA TABELA FATO: dw.FAT_RESULTADOS ---
✓ Tabela dw.FAT_RESULTADOS truncada para recarga.
✓ 26759 registros inseridos com sucesso em dw.FAT_RESULTADOS.
--- ✅ CARGA DO STAR SCHEMA FINALIZADA ---


___
### Finalizar conexão 

In [40]:
cursor.close()
conn.close()
print("✓ Conexão com o banco de dados fechada com sucesso!")
print("✓ ETL Silver → Gold concluído!")

✓ Conexão com o banco de dados fechada com sucesso!
✓ ETL Silver → Gold concluído!
